! jt -r

In [225]:
from importlib import reload
reload(aux)
reload(con)
reload(info)
reload(graph)

<module 'src.graphic' from 'C:\\Users\\ugur.eren\\Python Codes\\cefis2\\src\\graphic.py'>

In [226]:
import pandas as pd
import src.auxiliary_functions as aux
import src.condition as con
from datetime import time, datetime,date
import src.info as info
import src.in_out as io
import numpy as np
import itertools
import src.graphic as graph

# Hazırlık Aşaması

**data** klasöründeki tüm dosya yolları alınıyor
```python
>>> path_list
Out []:
 ['C:\\Users\\ugur.eren\\Python Codes\\cefis2\\data\\20180906_6A_U8.xlsx',
 'C:\\Users\\ugur.eren\\Python Codes\\cefis2\\data\\20180906_6B_U8.xlsx',
 'C:\\Users\\ugur.eren\\Python Codes\\cefis2\\data\\20180906_6C_U8.xlsx',
 'C:\\Users\\ugur.eren\\Python Codes\\cefis2\\data\\20180907_6A_U8.xlsx',
 'C:\\Users\\ugur.eren\\Python Codes\\cefis2\\data\\20180907_6B_U8.xlsx',
 'C:\\Users\\ugur.eren\\Python Codes\\cefis2\\data\\20180907_6C_U8.xlsx']
```

In [2]:
path_list = io.get_path('C:\\Users\\ugur.eren\\Python Codes\\cefis2\\data\\')

Ürün adları sırayla alınıyor
```py
>>> product_names
Out[]:
['6AU8', '6BU8', '6CU8', '6AU8', '6BU8', '6CU8']```

In [3]:
product_names = [info.get_productName(path) for path in path_list]

Tarihler sırayla alınıyor
```py
>>> date_list
Out[]:
[Timestamp('2018-09-06 00:00:00'),
 Timestamp('2018-09-06 00:00:00'),
 Timestamp('2018-09-06 00:00:00'),
 Timestamp('2018-09-07 00:00:00'),
 Timestamp('2018-09-07 00:00:00'),
 Timestamp('2018-09-07 00:00:00')]
```

In [4]:
info.get_productDate('C:\\Users\\ugur.eren\\Python Codes\\cefis2\\data\\20180906_6A_U8.xlsx').date()

datetime.date(2018, 9, 6)

In [5]:
date_list = [info.get_productDate(path) for path in path_list]

Hangi sütunlardaki verileri alacağımızı aşağıdaki listede belirityoruz

In [6]:
cols= ["Time","BID price","ASK price"]

Dosyalardan veriler dataframe olarak okunuyor ve listeye atılıyor.
Okunan_veriler'in yapısı: <br/>
```py
list(pd.DataFrame,pd.DataFrame.....)
```

In [7]:
okunan_veriler = [pd.read_excel(path,usecols=cols) for path in path_list]

Her bir DataFrame'deki sütunlar yeniden adlandırılıyor

In [8]:
columns=['time','bid_price','ask_price']

In [9]:
for df in okunan_veriler:
    df.columns = columns

In [10]:
time_stamp = okunan_veriler[0].time

# Mid Price Hesaplama ve Saatlik Bölme 

hesaplanan her bir mid_price **list_of_mid_price** adlı listeye atılıyor. Listenin yapısı aşağıdaki gibidir:
```py
list(pd.Series,pd.Series,pd.Series...)
```

In [11]:
list_of_mid_price = list(map(aux.get_mid_price,okunan_veriler))

Bulunan mid price hiyerarşik indeksleme ile tek DataFrame'de birleştiriliyor.

In [12]:
arrays = [date_list,product_names]

In [13]:
df_mp = pd.DataFrame(data=list_of_mid_price, index = arrays)

Bu dataframe zaman sütunu eklenip saatlik olarak bölünüyor.

In [14]:
df_mp_t = df_mp.T

In [15]:
df_mp_t['time'] = time_stamp

```py
>>> len(list_of_hourly_mp)
Out[]: 23
>>> df.columns.levels
Out[]:FrozenList([[2018-09-06 00:00:00, 2018-09-07 00:00:00, 'time'], ['6AU8', '6BU8', '6CU8', '']])
```

In [16]:
list_of_hourly_mp = aux.split_df(df_mp_t,23)

----

# Ürünler Eşleştiriliyor

```python
>>> non_duplicate_dates
Out[]:[Timestamp('2018-09-06 00:00:00'), Timestamp('2018-09-07 00:00:00')]
>>> non_duplicate_names
Out[]:['6AU8', '6BU8', '6CU8']
>>> paired_products
Out[]: [('6AU8', '6BU8'), ('6AU8', '6CU8'), ('6BU8', '6CU8')]
```

In [17]:
non_duplicate_dates = list(dict.fromkeys(date_list)) #tekrar eden tarihler kaldırıldı.

In [18]:
non_duplicate_names = list(dict.fromkeys(product_names)) #tekrar eden ürünler kaldırıldı.

In [19]:
paired_products = list(itertools.combinations(non_duplicate_names, 2)) #2'li kombinasyon kullanılarak ürünler eşleştiriliyor.

In [63]:
import plotly.express as px

----

# Spread Hesaplanıyor

**list_of_hourly_mp** da hiyerarşi şu şekildedir: <br/>
* Önce saate erişiriz
* İkinci olarak tarihe erişiriz
* Son olarak ürüne erişiriz <br/>
**Buna göre spread hesaplamak için aşağıdaki yol izlenir:** <br/>
    1. Her bir saate eriş
    2. Her bir tarihe eriş
    3. Pairleri al
    4. *spread_hesapla(pair): return pd.Series*
    

In [20]:
values={'a_PNLTICK':10,
'a_TICKSIZE':0.0001,
'b_PNLTICK':6.25,
'b_TICKSIZE':0.0001}

In [21]:
def spread_al(df):
    spread_dict = {}
    for date in non_duplicate_dates:
        for pair in paired_products:
            mp_1 = df[pd.Timestamp(date)][pair[0]]
            mp_2 = df[pd.Timestamp(date)][pair[1]]
            spread_dict[(date,pair[0]+"_"+pair[1])] = aux.find_spread(mp_1,mp_2,values)
    new_df = pd.DataFrame(spread_dict)
    new_df['time'] = df.time
    return new_df

In [22]:
spread = [spread_al(df) for df in list_of_hourly_mp]

In [84]:
import plotly.graph_objects as go

# Spread verisinin grafiği çiziliyor
Örnekte yalnızca 18:00:00 - 19:00:00 arası kullanıldı

In [211]:
spread_18 = spread[0]

In [232]:
AB = aux.extract_product(spread_18,'6AU8_6CU8')

In [294]:
def draw(df):
    fig = go.Figure()
    size = df.columns.size
    for i in range(size):
        data = df.iloc[:,i]
        name = df.columns.values.item(i).strftime("%m. %d. %Y")
        fig.add_trace(
            go.Scatter(
                x = df.index,
                y = data,
                name = name            
            )
        )
    return fig

In [295]:
sekil = draw(AB)

In [298]:
sekil.show()

In [ ]:
draw(AB)

In [284]:
AB.iloc[:,0]

time
18:00:00    0.000
18:00:01    0.000
18:00:02   -1.875
18:00:03   -1.875
18:00:04   -1.875
            ...  
18:59:55   -1.875
18:59:56   -1.875
18:59:57   -1.875
18:59:58   -1.875
18:59:59   -1.875
Name: 2018-09-06 00:00:00, Length: 3600, dtype: float64

In [287]:
AB.columns.values.item(0)

Timestamp('2018-09-06 00:00:00')

In [272]:
AB.columns.values.item(1).strftime("%m. %d. %Y")

'09. 07. 2018'

In [288]:
fig = go.Figure()

In [290]:
fig.add_trace(
    go.Scatter(
       # x = spread_18.time,
        y = AB.iloc[:,0],
        name = AB.columns.values.item(1).strftime("%m. %d. %Y")
    )
)

fig.add_trace(
    go.Scatter(
      #  x = spread_18.time,
        y = v3['6BU8_6CU8'][0:3600]
    )
)

In [151]:
fig2 = go.Figure()

In [153]:
fig2.add_trace(
    go.Scatter(
       # x = spread_18.time,
        y = [1,2,3,4]
    )
)

In [156]:
fig2.update_layout(xaxis_range=['2016-07-01','2016-7-5'])

In [150]:
fig.update_layout(xaxis_range=['2016-07-01','2016-12-31'],
                  title_text="Manually Set Date Range")

In [144]:
fig.show()

In [130]:
fig.update_layout(
                  title_text="Manually Set Date Range")

In [118]:
spread_18

2018-09-06 00:00:00                     2018-09-07 00:00:00            \
               6AU8_6BU8 6AU8_6CU8 6BU8_6CU8           6AU8_6BU8 6AU8_6CU8   
0                  0.000     0.000     0.000               0.000     0.000   
1                  0.000     0.000     0.000               0.000     0.000   
2                 -5.000    -1.875     3.125              -5.000    -1.875   
3                 -8.125    -1.875     8.125              -8.125    -1.875   
4                -11.250    -1.875    13.125             -11.250    -1.875   
...                  ...       ...       ...                 ...       ...   
3595               1.250    -1.875    -6.875               1.250    -1.875   
3596               1.250    -1.875    -6.875               1.250    -1.875   
3597               1.250    -1.875    -6.875               1.250    -1.875   
3598               1.250    -1.875    -6.875               1.250    -1.875   
3599               1.250    -1.875    -6.875               1.250    -1.875   

                    time  
     6BU8_6CU8            
0        0.000  18:00:00  
1        0.000  18:00:01  
2        3.125  18:00:02  
3        8.125  18:00:03  
4       13.125  18:00:04  
...        ...       ...  
3595    -6.875  18:59:55  
3596    -6.875  18:59:56  
3597    -6.875  18:59:57  
3598    -6.875  18:59:58  
3599    -6.875  18:59:59  

[3600 rows x 7 columns]

In [115]:
60*60

3600

In [117]:
v1.time[0:3600].to_csv('time.csv')

C:\Users\ugur.eren\AppData\Local\Continuum\anaconda3\envs\pandas\lib\site-packages\ipykernel_launcher.py:1: FutureWarning:

The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.



In [76]:
v1 = graph._split_product(spread_18)['6AU8_6BU8']

In [77]:
v2  = graph._split_product(spread_18)['6AU8_6CU8']

In [78]:
v3 = graph._split_product(spread_18)['6BU8_6CU8']

In [100]:
v3['6BU8_6CU8']

0        0.000
1        0.000
2        0.000
3        0.000
4        3.125
         ...  
3595   -23.125
3596   -23.125
3597   -23.125
3598   -23.125
3599   -23.125
Name: 6BU8_6CU8, Length: 3600, dtype: float64

,time,6BU8_6CU8
0,18:00:00,0.000
1,18:00:00,0.000
2,18:00:01,0.000
3,18:00:01,0.000
4,18:00:02,3.125
...,...,...
7195,18:59:57,-6.875
7196,18:59:58,-6.875
7197,18:59:58,-6.875
7198,18:59:59,-6.875


In [58]:
spread_18.stack().reset_index().set_index('level_1').drop('level_0',axis=1).loc['6AU8_6BU8']

,2018-09-06 00:00:00,2018-09-07 00:00:00,time
level_1,,,
6AU8_6BU8,0.000,0.000,NaN
6AU8_6BU8,0.000,0.000,NaN
6AU8_6BU8,-5.000,-5.000,NaN
6AU8_6BU8,-8.125,-8.125,NaN
6AU8_6BU8,-11.250,-11.250,NaN
...,...,...,...
6AU8_6BU8,1.250,1.250,NaN
6AU8_6BU8,1.250,1.250,NaN
6AU8_6BU8,1.250,1.250,NaN


In [64]:
px.data.iris()

,sepal_length,sepal_width,petal_length,petal_width,species,species_id
0,5.1,3.5,1.4,0.2,setosa,1
1,4.9,3.0,1.4,0.2,setosa,1
2,4.7,3.2,1.3,0.2,setosa,1
3,4.6,3.1,1.5,0.2,setosa,1
4,5.0,3.6,1.4,0.2,setosa,1
...,...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,virginica,3
146,6.3,2.5,5.0,1.9,virginica,3
147,6.5,3.0,5.2,2.0,virginica,3
148,6.2,3.4,5.4,2.3,virginica,3
